
# <font color=blue><div align="center">ST7 SNCF : Jalon 1</div></font>

## <font color=blue><div align="center">"<em>Indian Railways</em>"</div></font>

## Modules

In [ ]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
%matplotlib inline

# Module relatif à Gurobi
from gurobipy import *

# Module csv
import csv

In [ ]:
mini_instance="mini_instance.xlsx"
instance_simple="instance_WPY_simple.xlsx"
instance_realiste="instance_WPY_realiste_jalon1.xlsx"

# CHOIX DE L'INTSANCE 
instance=mini_instance

## Traitement de données

In [ ]:
def excel_to_dict(file :str)-> dict[str]:
    '''
    Takes excel file name in the repo
    and returns a dictionary[sheet_name]=Dataframe
    '''
    # Load the Excel file
    xl = pd.ExcelFile(file)
    
    # Get the names of the sheets
    sheet_names = xl.sheet_names
    
    # Initialize a dictionary to store the data
    data_dict = {}
    
    # Iterate over each sheet
    for sheet_name in sheet_names:
        # Read the sheet into a pandas DataFrame
        data_dict[sheet_name] = pd.read_excel(file, sheet_name=sheet_name)
    return data_dict

def date_to_creneau(date0: datetime.datetime, hour: pd.DataFrame, date: pd.DataFrame):
    '''
    Takes hour and date's 1-column dataframes
    and returns a dataframe of 1-column of 'créneaux'
    starting from date0
    '''
    # Excel is sometimes stoopid and has inconsistent types
    if type(date[0])==str:
        datetime_df=pd.to_datetime(date.apply(lambda x: str(x)+' ')+hour.apply(lambda x:str(x)), dayfirst=True)
    if type(date[0])==pd._libs.tslibs.timestamps.Timestamp:
        datetime_df=pd.to_datetime(date.apply(lambda x: str(x)+' ')+hour.apply(lambda x:str(x)))
    return (datetime_df-date0).dt.total_seconds().div(60*15).astype(int)+1

def creneau_to_date(date0: datetime.datetime, creneau : int):
    return date0+ datetime.timedelta(minutes=(creneau-1)*15)

def convToStr(date)-> str:
    '''
    Takes a date in any form in the instance
    and returns a string dd/mm/yyyy
    '''
    if type(date)==str:
        return date
    else:
        return "/".join(str(date.date()).split("-")[::-1])
    

INSTANCE= excel_to_dict(instance)

### Dictionnaires utiles 

Trains de l'arrivée et trains du départ

In [ ]:
trains_arrivee=[]
for index, row in INSTANCE["Sillons arrivee"].iterrows():
    id_train = row['n°TRAIN'], convToStr(row["JARR"])
    trains_arrivee.append(id_train)

trains_depart=[]
for index, row in INSTANCE["Sillons depart"].iterrows():
    id_train = row['n°TRAIN'], convToStr(row["JDEP"])
    trains_depart.append(id_train)

Créneaux de l'arrivée et du départ

In [ ]:
# Date de référence est celle du lundi de la première semaine
# Ce choix rend facile la traduction des indisponibilités de type (weekday,hh:mm)
date0=str(INSTANCE["Sillons arrivee"]["JARR"][0])+' 00:00:00'
date0=pd.to_datetime(date0, dayfirst=True)
date0=date0-datetime.timedelta(days=float(date0.dayofweek))
datef=pd.to_datetime(INSTANCE["Sillons depart"]["JDEP"].iloc[-1], dayfirst=True)+datetime.timedelta(hours=24)
nombre_creneaux=int((datef-date0).total_seconds()/(60*15))

# Créneaux du départ et de l'arrivée
INSTANCE["Sillons arrivee"]["creneau"]=date_to_creneau(date0, INSTANCE["Sillons arrivee"]["HARR"], INSTANCE["Sillons arrivee"]["JARR"])
INSTANCE["Sillons depart"]["creneau"]=date_to_creneau(date0, INSTANCE["Sillons depart"]["HDEP"], INSTANCE["Sillons depart"]["JDEP"])

# Dictionnaires des créneaux 
# { (n°train, jour) : créneau }
creneau_arrivee={} 
for index, row in INSTANCE["Sillons arrivee"].iterrows():
    creneau_arrivee[row["n°TRAIN"],convToStr(row["JARR"])]=row["creneau"]

creneau_depart={}
for index, row in INSTANCE["Sillons depart"].iterrows():
    creneau_depart[row["n°TRAIN"],convToStr(row["JDEP"])]=row["creneau"]

Correspondances

In [ ]:
def correspondances():
    '''
    Construit un dictionnaire global
    associant chaque train du départ à un set 
    des trains d'arrivée qui le constituent
    et à un train d'arrivée un set de trains 
    de départs qui lui y sont liés
    '''
    At={}
    Dt={}
    for train in trains_depart:
        At[train]=set()
    for train in trains_arrivee:
         Dt[train]=set()
    for index, row in INSTANCE["Correspondances"].iterrows():
            At[(row["n°Train depart"], convToStr(row["Jour depart"]))].add((row['n°Train arrivee'],convToStr(row['Jour arrivee'])))
            Dt[(row["n°Train arrivee"], convToStr(row["Jour arrivee"]))].add((row['n°Train depart'],convToStr(row['Jour depart'])))
            
    return At,Dt

At=correspondances()[0]
Dt=correspondances()[1]


Indisponibilité des machines et des chantiers

In [ ]:
def creneaux_de_periode(indispo_str : str)-> list[int]: 
    plage_jours=nombre_creneaux//96
    creneaux = []
    if indispo_str!='0':
        periodes = indispo_str.split(';')
        for periode in periodes:
            jour, plage_horaire = periode.strip('()').split(',')
            jour = int(jour)
            jours=[jour+7*k for k in range(0,plage_jours//7+1)]
            plage_horaire = plage_horaire.strip('()').split('-')
            heure_debut, minute_debut = map(int, plage_horaire[0].split(':'))
            creneau_debut=minute_debut//15+1
            heure_fin, minute_fin = map(int, plage_horaire[1].split(':'))
            creneau_fin=minute_fin//15+1


            if (heure_debut,minute_debut)>=(heure_fin,minute_fin):
                for jour1 in jours:
                    for c in range((jour1-1)*96+heure_debut*4+creneau_debut,jour1*96):
                        if c<=nombre_creneaux:
                            creneaux.append(c)
                    for c in range(jour1*96,jour1*96+heure_fin*4+creneau_fin):
                        if c<=nombre_creneaux: creneaux.append(c)
            else:
                for jour1 in jours:
                    for c in range((jour1-1)*96+heure_debut*4+creneau_debut,(jour1-1)*96+heure_fin*4+creneau_fin):
                        if c<=nombre_creneaux: creneaux.append(c) 
    return creneaux

# data1 = INSTANCE1['Machines']
df = INSTANCE['Machines']

df['Indisponibilites']=df['Indisponibilites'].astype(str)
# Apply the function to each row
df['Indisponibilites_TimeSlots'] = df['Indisponibilites'].apply(creneaux_de_periode)

DEB_INDIS = df.loc[df['Machine'] == 'DEB', 'Indisponibilites_TimeSlots'].values[0]
FOR_INDIS = df.loc[df['Machine'] == 'FOR', 'Indisponibilites_TimeSlots'].values[0]
DEG_INDIS = df.loc[df['Machine'] == 'DEG', 'Indisponibilites_TimeSlots'].values[0]

In [ ]:
df=INSTANCE["Chantiers"]
df['Indisponibilites']=df['Indisponibilites'].astype(str)
df['Indisponibilites_TimeSlots'] = df['Indisponibilites'].apply(creneaux_de_periode)

REC_INDIS = df.loc[df['Chantier'] == 'WPY_REC', 'Indisponibilites_TimeSlots'].values[0]
OFORM_INDIS = df.loc[df['Chantier'] == 'WPY_FOR', 'Indisponibilites_TimeSlots'].values[0]
DEP_INDIS = df.loc[df['Chantier'] == 'WPY_DEP', 'Indisponibilites_TimeSlots'].values[0]

NVREC=df.loc[df['Chantier'] == 'WPY_REC', "Nombre de voies"].values[0]
NVFOR=df.loc[df['Chantier'] == 'WPY_FOR', "Nombre de voies"].values[0]
NVDEP=df.loc[df['Chantier'] == 'WPY_DEP', "Nombre de voies"].values[0]

In [ ]:
epsilon=0.00001
M=100000

## Modèle mathématique 
On a modélisé notre problème en partant du fait que les machines ne sont disponibles que pour un seul train et dans un seul créneau. \
De ce fait, on introduit les variables *binaires* suivantes : \
\
    - $(deb_{ic})$ : variable binaire indiquant si la machine de débranchement est utilisée \
    pour le sillion d'arrivée d'identifiant $i$ au créneau $c$ \
    - $(for_{ic})$ : variable binaire indiquant si la machine de formation est utilisée \
    pour le sillion du départ d'identifiant $i$ au créneau $c$ \
    - $(deg_{ic})$ : variable binaire indiquant si la machine de dégarage est utilisée \
    pour le sillion du départ d'identifiant $i$ au créneau $c$

Les créneaux ont une durée de $15 min$ correspondant à la durée d'une tâche machine. Les valeurs possibles des créneaux dépendent de l'instance, \
soit alors $c \in \{1,2,..., c_{max}\} $ où $c=1$ correspond au premier créneau du lundi de la première semaine du travail : $[0:00 , 0:15[$ et \
$c_{max}$ le dernier créneau du dernier jour du travail : $[23:45, 0:00[$.

On note ainsi : 
$$A = \{\text{Les identifiants des trains d'arrivée}\}$$
$$D = \{\text{Les identifiants des trains du départ}\}$$
$$C= \{1,2,..., c_{max}\}$$

In [ ]:
# IMPLEMNTATION DES VARIABLES 
# Implémentation Python
m= Model('Modèle')
deb = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'deb_{i}_{c}') for i in trains_arrivee for c in range(1,nombre_creneaux+1)}
form = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'deb_{i}_{c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}
deg = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'deg_{i}_{c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}

Pour inclure les informations sur les voies, nous avons défini les variables binaires suivantes:\
\
    - $orec_{ic}$ : variable binaire indiquant si le train d'arrivée $i \in A$ occupe une voie dans le chantier de débranchement au créneau $c \in C$.\
    - $ofor_{ic}$ : variable binaire indiquant si le train de départ $i \in D$ occupe une voie dans le chantier de formation au créneau $c \in C$.\
    - $odep_{ic}$ : variable binaire indiquant si le train de départ $i \in D$ occupe une voie dans le chantier de départ au créneau $c \in C$.
Les créneaux de $C$, et les ensembles $A$ et $D$ sont définis comme dans le jalon précédent.

In [ ]:
orec = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'orec_{i}_{c}') for i in trains_arrivee for c in range(1,nombre_creneaux+1)}
oform = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'oform_{i}_{c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}
odep = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'odep_{i}_{c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}

## Contraintes
### 1. Unique débranchement, formation et dégarage
Chaque train d'arrivé subit un seul et unique débranchement, chaque train du départ subit un seul et unique débranchement et dégarage. 
$$ \forall i \in A \quad \sum_{c} deb_{i,c} = 1 $$
$$ \forall i \in D \quad \sum_{c} for_{i,c} = 1 $$
$$ \forall i \in D \quad \sum_{c} deg_{i,c} = 1 $$

In [ ]:
uniqueDeb_train = {i : m.addConstr(quicksum([deb[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueDeb_train{i}') for i in trains_arrivee}
uniqueform_train = {i : m.addConstr(quicksum([form[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueFor_train{i}') for i in trains_depart}
uniqueDeg_train = {i : m.addConstr(quicksum([deg[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueDeg_train{i}') for i in trains_depart}

### 2. Une seule tâche machine par créneau  
Chaque machine est exploitée par au plus un seul train sur chaque créneau.
$$ \forall c \in C \quad \sum_{i \in A} deb_{i,c} \leq 1 $$
$$ \forall c \in C \quad \sum_{i \in D} for_{i,c} \leq 1 $$
$$ \forall c \in C \quad \sum_{i \in D} deg_{i,c} \leq 1 $$

In [ ]:
uniqueDeb_creneau = {c : m.addConstr(quicksum([deb[(i, c)] for i in trains_arrivee]) <=1, name = f'UniqueDeb_creneau{c}') for c in range(1, nombre_creneaux + 1)}
uniqueFor_creneau = {c : m.addConstr(quicksum([form[(i, c)] for i in trains_depart]) <=1, name = f'UniqueFor_creneau{c}') for c in range(1, nombre_creneaux + 1)}
uniqueDeg_creneau = {c : m.addConstr(quicksum([deg[(i, c)] for i in trains_depart ]) <=1, name = f'UniqueDeg_creneau{c}') for c in range(1, nombre_creneaux + 1)}

### 3. Débranchement se fait après l'arrivée du train
Pour chaque train d'arrivée $i$, le débranchement ne peut pas avoir lieu dans les créneaux précedent celui d'arrivée $c^{a}_{i}$ après une heure de tâches humaines.
$$ \forall i \in A \quad \sum_{c \space \leq \space c^{a}_{i}+4} deb_{i,c} = 0$$ 

In [ ]:
deb_apres_arrivee = {i : m.addConstr(quicksum([deb[(i, c)] for c in range(1, creneau_arrivee[i] + 5)]) ==0, name = f'deb_apres_arrivee{i}') for i in trains_arrivee}


### 4. Formation d'un train de départ 
Pour chaque train du départ $t$, la formation ne peut avoir lieu qu'après débranchement de tous les trains ayant les wagons qui le constituent. \
On note : $$ \forall t \in D \quad A_t = \{\text{Les identifiants des trains d'arrivée ayant au moins un wagon du train t}\}$$
On peut exprimer la contrainte comme suit : 
$$ \forall t \in D, \space\forall c \in C, \space\forall i \in A_t \quad for_{t,c} \leq \sum_{c' \space \lt \space c} deb_{i,c'}$$ 

In [ ]:
for_apres_deb = {(t,c,i) : m.addConstr(form[(t,c)]<=quicksum([deb[(i, c_p)] for c_p in range(1, c)]), name = f'for_apres_deb{t}{c}{i}') for t in trains_depart for c in range(1,nombre_creneaux+1) for i in At[t]}

### 5. Dégarage après formation
Pour chaque train du départ, le dégarage ne peut avoir lieu qu'après la formation du train et $150$ minutes soit $10$ créneaux de tâches humaines.
$$ \forall t \in D, \space\forall c \in C \quad deg_{t,c} \leq \sum_{c' \space \lt \space c-10} for_{t,c'}$$ 

In [ ]:
deg_apres_for = {(t,c) : m.addConstr(deg[(t,c)]<=quicksum([form[(t, c_p)] for c_p in range(1, c-9)]), name = f'deg_apres_for{t},{c}') for t in trains_depart for c in range(1,nombre_creneaux+1)}

### 6. Dégarage avant départ
Pour chaque train du départ $t$, le dégarage ne peut pas avoir lieu dans les créneaux suivants \
celui de $20$ minutes (de tâches humaines) avant le créneau du départ $c^{d}_{t}$.
$$ \forall t \in D \quad \sum_{c \space \geq \space c^{d}_{t}-2} deg_{t,c} = 0$$ 

In [ ]:
deg_avant_depart = {t : m.addConstr(quicksum([deg[(t, c)] for c in range(creneau_depart[t]-2, nombre_creneaux+1)])==0, name = f'depart_apres_deg{t}') for t in trains_depart}

### 7. Disponibilités des machines
Il y a des créneaux où les machines sont indisponible, soient alors  $ \space C_{deb}, C_{for}, C_{deg} \space$  ces crénaux d'indisponibilités.
$$ \forall i \in A \quad \sum_{c \space \in \space C_{deb}} deb_{i,c} = 0 $$
$$ \forall i \in D \quad \sum_{c \space \in \space C_{for}} for_{i,c} = 0 $$
$$ \forall i \in D \quad \sum_{c \space \in \space C_{deg}} deg_{i,c} = 0 $$

In [ ]:
indisponibilite_deb = {i : m.addConstr(quicksum([deb[(i, c)] for c in DEB_INDIS]) ==0, name = f'indisponibilité_deb{i}') for i in trains_arrivee }
indisponibilite_for = {i : m.addConstr(quicksum([form[(i, c)] for c in FOR_INDIS]) ==0, name = f'indisponibilité_for{i}') for i in trains_depart}
indisponibilite_deb = {i : m.addConstr(quicksum([deg[(i, c)] for c in DEG_INDIS]) ==0, name = f'indisponibilité_deg{i}') for i in trains_depart}

### Contraintes Jalon 2

#### Nombre des voies disponibles
Pour chaque créneau $c \in C$, un chantier ne peut pas accueillir plus de trains que le nombre de voies disponibles: 
$$ \forall c \in C \quad \sum_{i \in A} orec_{i,c} \leq \text{NVREC} $$
$$ \forall c \in C \quad \sum_{i \in D} ofor_{i,c} \leq \text{NVFOR} $$
$$ \forall c \in C \quad \sum_{i \in D} odep_{i,c} \leq \text{NVDEP} $$

Où NVDEB, NVFOR, NVDEG correspondent aux nombres de voies disponibles pour chaque chantiers REC, FOR,
DEP respectivement. \

In [ ]:
Nb_voies_rec = {c : m.addConstr(quicksum([orec[(i, c)] for i in trains_arrivee]) <= NVREC, name = f'nb_voies_rec{c}') for c in range(1,nombre_creneaux+1) }
Nb_voies_for = {c : m.addConstr(quicksum([oform[(i, c)] for i in trains_depart]) <= NVFOR, name = f'nb_voies_form{c}') for c in range(1,nombre_creneaux+1) }
Nb_voies_dep = {c : m.addConstr(quicksum([odep[(i, c)] for i in trains_depart]) <= NVDEP, name = f'nb_voies_dep{c}') for c in range(1,nombre_creneaux+1) }

Ainsi, il y a des créneaux dans lesquels les chantiers sont indisponibles qu'on note $C_{rec}, C_{for}, C_{dep}$. 
On interdit alors l'occupation lors de ces créneaux : 

$$ \forall i \in A \quad \sum_{c \space \in \space C_{rec}} orec_{i,c} = 0 $$
$$ \forall i \in D \quad \sum_{c \space \in \space C_{for}} ofor_{i,c} = 0 $$
$$ \forall i \in D \quad \sum_{c \space \in \space C_{dep}} odep_{i,c} = 0 $$

In [ ]:
indisponibilite_orec = {i : m.addConstr(quicksum([orec[(i, c)] for c in REC_INDIS]) ==0, name = f'indisponibilité_rec{i}') for i in trains_arrivee }
indisponibilite_oform = {i : m.addConstr(quicksum([oform[(i, c)] for c in OFORM_INDIS]) ==0, name = f'indisponibilité_oform{i}') for i in trains_depart}
indisponibilite_dep = {i : m.addConstr(quicksum([odep[(i, c)] for c in DEP_INDIS]) ==0, name = f'indisponibilité_dep{i}') for i in trains_depart}

#### Occupation du chantier de la réception pour un train d'arrivée
Un train d'arrivée $i \in A$ occupe le chantier de débranchement à partir de son créneau d'arrivée et jusqu'au créneau de son débranchement.
$$
    \forall c \in C, \forall i \in A : 
    (c\geq c_a^{i}) \land (\sum\limits_{c_1 \leq c} deb_{i,c_1}=0 ) \Leftrightarrow orec_{i,c}=1
$$
Qu'on linéarise comme suit:
$$
    \epsilon + M (-1+ orec_{i,c}) \leq (1- \sum\limits_{c_1 \leq c} deb_{i,c_1}) (c- c_a^{i}) \leq M . orec_{i,c}
$$
et 
$$orec_{i,c^i_{a}}=1$$
Où $M$ est un majorant suffisament grand et $\epsilon$ un minorant assez petit.

In [ ]:
occupation_rec_ub = {(i,c) : m.addConstr(epsilon+M*(-1+orec[(i,c)])<=(1-quicksum([deb[(i, c_1)] for c_1 in range(1, c+1)]))*(c-creneau_arrivee[i]), name = f'occupation_rec_ub{i,c}') for i in trains_arrivee for c in range(1,nombre_creneaux+1)}
occupation_rec_lb = {(i,c) : m.addConstr((1-quicksum([deb[(i, c_1)] for c_1 in range(1, c+1)]))*(c-creneau_arrivee[i])<=M*orec[(i,c)], name = f'occupation_rec_lb{i,c}') for i in trains_arrivee for c in range(1,nombre_creneaux+1)}
occupation_rec_creneau_arrivee={i: m.addConstr(orec[(i,creneau_arrivee[i])]==1,name='occupation_rec_creneau_arrivee{i}') for i in trains_arrivee}


#### Liens entre chantiers de débranchement et de formation
Pour un train d'arrivée $t \in A$, la machine de débranchement le débranche et renvoie ses wagones vers le chantier de formation dans le même créneau. 
On définit pour un train d'arrivée $t \in A$, l'ensemble $D_t$ des trains de départ associés aux wagons de $t$. 
$$ \forall t \in A \quad D_t = \{\text{Les identifiants des trains du départ ayant au moins un wagon du train t}\}$$
Il faut alors garantir: 
$$
    \forall t \in A, \forall c \in C \quad deb_{t,c}=1 \Rightarrow \prod\limits_{i \in D_t} ofor_{i,c} =1
$$
qu'on linéarise comme suit:
$$
    \forall t \in A, \forall i \in D_t, \forall c \in C \quad deb_{t,c} \leq ofor_{i,c}
$$

In [ ]:
lien_rec_for = {(t,c,i) : m.addConstr(deb[(t,c)]<=oform[(i,c)], name = f'lien_rec_for{t}{c}{i}') for t in trains_arrivee for c in range(1,nombre_creneaux+1) for i in Dt[t]}

Il faut aussi garantir que pendant le créneau de débranchement, le train d'arrivée $t$ occupe le chantier de la réception: 
$$
    \forall t \in A, \forall c \in C \quad deb_{t,c} \leq orec_{t,c}
$$

In [ ]:
occupe_rec_for = {(t,c) : m.addConstr(deb[(t,c)]<=orec[(t,c)], name = f'occupe_rec_for{t}{c}') for t in trains_arrivee for c in range(1,nombre_creneaux+1)}

#### Débranchement et Disponibilités des voies
Lorsqu'un train d'arrivée est débranché, il occupe immédiatement les voies réservées aux trains du départs qu'il constitue. Il en découle qu'on ne peut pas débrancher un train $t \in A$ au créneau $c \in C$ si au créneau $c-1$ les voies supplémentaires 
nécessaires n'étaient pas disponibles : 
$$\text{Card}\{i \in D_t \text{\space t.q \space} ofor_{i,c-1}=0\} > \text{Card}\{\text{voies dispo. à $c-1$}\} \Longrightarrow deb_{t,c}=0$$ 
Notant pour simplicité :
$$\forall t \in A, \forall c \ge 2 \quad \text{for\_libres}_{c-1}=\text{NVFOR}-\sum_{u \in D}ofor_{u,c-1}$$
Ce qu'on traduit par la contrainte linéaire suivante :
$$ \forall t \in A, \space \forall c \geq 2 \quad \text{for\_libres}_{c-1}-\sum_{i \in D_t} \left(1-ofor_{i,c-1}\right)
\le \text{M} \left(1-deb_{t,c}\right)$$
Où on peut prendre M $=$ NVFOR $+1$.

In [ ]:
deb_dispos_voies = {(t,c) : m.addConstr(NVFOR-quicksum([oform[(u, c-1)] for u in trains_depart])-quicksum([1-oform[(i, c-1)] for i in Dt[t]])<=M*(1-deb[(t,c)]), name = f'deb_dispos_voies{t,c}') for t in trains_arrivee for c in range(2,nombre_creneaux+1)}


#### Occupation du chantier de formation
Un train du départ $t \in D$ occupe le chantier de formation dès le débranchement d'un train $i \in D_t$ et jusqu'à son dégarage. 
Vu qu'on a déja imposé une occupation d'une voie pour $t$ durant le créneau de déformation de tout train $i \in D_t$, nous avons pensé à la formulation suivante:\

$$\forall t \in D, \forall c \in C \quad ofor_{tc}=1 \Rightarrow  \prod\limits_{c \leq c_1\leq c^*}  ofor_{tc_1}=1,$$ 
t.q $$deg_{tc^*}=1$$

Nous avons linéarisé cette contrainte comme suit:
$$\forall c \in C, \forall c_1 \geq c \quad ofor_{tc}. \sum \limits_{c_2 \geq c_1} deg_{tc_2} \leq ofor_{tc_1}$$

In [ ]:
occupe_form = {(t,c,c1) : m.addConstr(oform[(t,c)]*quicksum([deg[(t,c2)] for c2 in range(c1,nombre_creneaux+1)])<=oform[(t,c1)], name = f'occupe_form{t,c,c1}') for t in trains_depart for c in range(1,nombre_creneaux+1) for c1 in range(c,nombre_creneaux+1)}


#### Gestion des voies entre les chantiers formation et départ
Déja, comme auparavant, quand on dégare un train de départ $t \in D$, on lui donne une voie dans le chantier de départ:
$$ \forall t \in D, \forall c \in C \quad deg_{tc} = 1 \Rightarrow odep_{tc}=1 $$
Qui est linéarisée comme suit:
$$ \forall t \in D, \forall c \in C \quad deg_{tc} \leq odep_{tc}$$

In [ ]:
occupe_dep_qd_deg = {(t,c) : m.addConstr(deg[(t,c)]<=odep[(t,c)], name = f'occupe_dep_qd_deg{t}{c}') for t in trains_depart for c in range(1,nombre_creneaux+1)}

La deuxième contrainte dans cette section est, pour dégarer un train de départ $t \in D$ dans un créneau $c$, il faut vérifier qu'il existe une voie disponible dans laquelle on peut le dégarer pendant le créneau $c-1$.
Il faut donc:
Pour $$\forall t \in D, \forall c \ge 2 \quad \text{deg\_libres}_{c-1}=\text{NVDEP}-\sum_{u \in D}ofor_{u,c-1}$$
Il faut:
$$\text{deg\_libres}_{c-1}=0 \Rightarrow deg_{t,c} =0$$
Qu'on linéarise par:
$$deg_{t,c} \leq \text{deg\_libres}_{c-1}$$

In [ ]:
verif_avant_deg = {(t,c) : m.addConstr(deg[(t,c)]<=NVDEP-quicksum([oform[(u,c-1)] for u in trains_depart]), name = f'verif_avant_deg{t,c}') for t in trains_depart for c in range(2,nombre_creneaux+1)}


Ainsi, un train du départ occupe le chantier après dégarage et avant le départ : 
$$
    \forall c \in C, \forall i \in D : 
    (c\leq c_d^{i}) \land (\sum\limits_{c_1 \leq c} deg_{i,c_1}=1 ) \Leftrightarrow odep_{i,c}=1
$$
Qu'on linéarise comme suit:
$$
    \epsilon + M (-1+ odep_{i,c}) \leq  (\sum\limits_{c_1 \leq c} deg_{i,c_1}) ( c_d^{i}-c) \leq M . odep_{i,c}
$$
et 
$$odep_{i,c^i_{d}}=1$$

In [ ]:
occupation_dep_ub = {(i,c) : m.addConstr(epsilon+M*(-1+odep[(i,c)])<=(quicksum([deg[(i, c_1)] for c_1 in range(1, c+1)]))*(creneau_depart[i]-c), name = f'occupation_dep_ub{i,c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}
occupation_dep_lb = {(i,c) : m.addConstr((quicksum([deg[(i, c_1)] for c_1 in range(1, c+1)]))*(creneau_depart[i]-c)<=M*orec[(i,c)], name = f'occupation_dep_lb{i,c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}
occupation_rec_creneau_arrivee={i: m.addConstr(odep[(i,creneau_depart[i])]==1, name=f'occupation_rec_creneau_arrivee{i}') for i in trains_depart}


## Résolution

In [ ]:
objfunction = quicksum([oform[(t,c)] for t in trains_depart for c in range(1,nombre_creneaux+1)])
m.setObjective(objfunction , GRB.MINIMIZE)
m.params.outputflag = 0
m.update()

In [ ]:
m.optimize()
if m.status == GRB.INF_OR_UNBD:
    m.setParam(GRB.Param.Presolve, 0)
    m.optimize()

if m.status == GRB.INFEASIBLE:
    print(m.display(), "\n\tN'A PAS DE SOLUTION!!!")
elif m.status == GRB.UNBOUNDED:
    print(m.display(), "\n\tEST NON BORNÉ!!!")

In [ ]:
# outputxlsx = "output.xlsx"
# output = {'Id tâche': [], 'Type de tâche' : [], 'Jour' : [], 'Heure début' : [], 'Durée' : [], 'Sillon' : []}
# for train in trains_arrivee:
#     print(train)
#     print('--------------------------------------------')
#     for c in range(1,nombre_creneaux+1):
#         if deb[(train,c)].x==1:
#             output['Id tâche'].append(f'DEB_{train[0]}_'+ creneau_to_date(date0, c).strftime('%d/%m/%Y'))
#             output['Type de tâche'].append('DEB')
#             output['Jour'].append(creneau_to_date(date0, c).strftime('%d/%m/%Y'))
#             output['Heure début'].append(creneau_to_date(date0, c).strftime('%H:%M'))
#             output['Durée'].append(15)
#             output['Sillon'].append(train[0])
#             print('deb de {} se fait au créneau :'.format(train))
#             print(creneau_to_date(date0,c),' - ',creneau_to_date(date0,c+1))
#             print('-------------------------------------------------------')


# for train in trains_depart:
#     print(train)
#     print('--------------------------------------------')
#     for c in range(1,nombre_creneaux+1):
#         if form[(train,c)].x==1:
#             print('For de {} se fait au créneau :'.format(train))
#             output['Id tâche'].append(f'FOR_{train[0]}_'+ creneau_to_date(date0, c).strftime('%d/%m/%Y'))
#             output['Type de tâche'].append('FOR')
#             output['Jour'].append(creneau_to_date(date0, c).strftime('%d/%m/%Y'))
#             output['Heure début'].append(creneau_to_date(date0, c).strftime('%H:%M'))
#             output['Durée'].append(15)
#             output['Sillon'].append(train[0])
#             print(creneau_to_date(date0,c),' - ',creneau_to_date(date0,c+1))
#             print('-------------------------------------------------------')
#         if deg[(train,c)].x==1:
#             print('deg de {} se fait au créneau :'.format(train))
#             output['Id tâche'].append(f'DEG_{train[0]}_' + creneau_to_date(date0, c).strftime('%d/%m/%Y'))
#             output['Type de tâche'].append('DEG')
#             output['Jour'].append(creneau_to_date(date0, c).strftime('%d/%m/%Y'))
#             output['Heure début'].append(creneau_to_date(date0, c).strftime('%H:%M'))
#             output['Durée'].append(15)
#             output['Sillon'].append(train[0])
#             print(creneau_to_date(date0,c),' - ',creneau_to_date(date0,c+1))
#             print('-------------------------------------------------------')

# # Piping output into an excel sheet
# outputdf=pd.DataFrame(output)
# outputdf.to_excel(outputxlsx, index=False)

In [ ]:
print('test')